In [1]:
import pandas as pd
import numpy as np
import joblib
import warnings



warnings.filterwarnings('ignore')

In [2]:

def preprocess_data(data):

    preprocessor = joblib.load('assets/models/preprocessor.pkl')

    #le = preprocessor['label_encoder'] 

    le_state = preprocessor['label_encoder_state']
    le_crop = preprocessor['label_encoder_crop']
    le_target = preprocessor['label_encoder_target']

    target_enc = preprocessor['target_encoder']

    #scaler = preprocessor['scaler']

    features = preprocessor['features']


    df = pd.DataFrame([data])

    df = df.rename(columns={'Leaf wetness probability_mean': 'Leaf wetness probability_mean_filled'})
    
    df['state'] = le_state.transform(df['state'])
    df['crop'] = le_crop.transform(df['crop'])  

    df[['district', 'subdistrict']] = target_enc.transform(df[['district', 'subdistrict']])
    
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month

    

    New_df = df[features].copy()

   
    #New_df = scaler.transform(New_df)
    
    

    return New_df,le_target


In [3]:


def predict_disease(data,model_name):

    new_df, le_target = preprocess_data(data)

    #print(new_df.columns)

    if model_name == 'xgb':
        
        xg_model = joblib.load('assets/models/xgb_smote_pipeline.pkl')
        pred_proba = xg_model.predict(new_df)
        pred_label = le_target.inverse_transform([pred_proba])[0]


    elif model_name == 'dl':
        from tensorflow.keras.models import load_model
        
        keras_model = load_model('assets/models/Deeplearning_class_weight_model.keras')

        scaler = joblib.load('assets/models/scaler.pkl')
        new_df = scaler.transform(new_df)

        

        pred_proba = keras_model.predict(new_df)
        pred_class = pred_proba.argmax(axis=1)  # get class index
        pred_label = le_target.inverse_transform([pred_class])[0]
    
    
    return pred_label

In [4]:
input_1 = {
    'date': '01-06-2023',
    'state': 'Telangana',
    'district': 'Nalgonda',
    'subdistrict': 'Marriguda',
    'crop': 'TOMATO',
    'peat_id': 600022,
    'Leaf wetness probability_mean': 1.207049629,
    'Precipitation Total_mean': 0.004761905,
    'Relative Humidity_max': 65.05858243,
    'Relative Humidity_min': 32.70756829,
    'Soil Moisture_mean': 0.145011909,
    'Soil Temperature_mean': 34.23577343,
    'Sunshine Duration_mean': 23.388835,
    'Temperature_max': 43.29,
    'Temperature_min': 27.05,
    'Actual_Disease': 'Pest_D'
    }


input_2 = {
    'date': '01-06-2023',
    'state': 'Maharashtra',
    'district': 'Nashik',
    'subdistrict': 'Kalwan',
    'crop': 'TOMATO',
    'peat_id': 200036,
    'Leaf wetness probability_mean': 0.052521027,
    'Precipitation Total_mean': 0.002380953,
    'Relative Humidity_max': 84.17283286,
    'Relative Humidity_min': 33.22324771,
    'Soil Moisture_mean': 0.188488103,
    'Soil Temperature_mean': 31.328335,
    'Sunshine Duration_mean': 27.65686014,
    'Temperature_max': 43.86428571,
    'Temperature_min': 22.11285714,
    'Actual_Disease': 'Pest_A'
}



input_3 = {
    'date': '01-06-2023',
    'state': 'Maharashtra',
    'district': 'Gondiya',
    'subdistrict': 'Deori',
    'crop': 'RICE',
    'peat_id': 300032,
    'Leaf wetness probability_mean': 17.38573794,
    'Precipitation Total_mean': 0.049404763,
    'Relative Humidity_max': 72.73967043,
    'Relative Humidity_min': 32.14477529,
    'Soil Moisture_mean': 0.286916664,
    'Soil Temperature_mean': 31.44029643,
    'Sunshine Duration_mean': 30.09109686,
    'Temperature_max': 42.21571429,
    'Temperature_min': 23.65,
    'Actual_Disease': 'Pest_B'
}

input_4 = {
    'date': '13-11-2024',
    'state': 'Telangana',
    'district': 'Bhadradri',
    'subdistrict': 'Karakagudem',
    'crop': 'RICE',
    'peat_id': 300032,
    'Leaf wetness probability_mean': 18.32679867,
    'Precipitation Total_mean': 0.029166668,
    'Relative Humidity_max': 87.32765833,
    'Relative Humidity_min': 48.38298333,
    'Soil Moisture_mean': 0.2472639,
    'Soil Temperature_mean': 28.03972333,
    'Sunshine Duration_mean': 18.04236833,
    'Temperature_max': 34.89666667,
    'Temperature_min': 22.37,
    'Actual_Disease': 'Pest_B'
}
input_5 = {
    'date': '13-11-2024',
    'state': 'Maharashtra',
    'district': 'Satara',
    'subdistrict': 'Karad',
    'crop': 'TOMATO',
    'peat_id': 300050,
    'Leaf wetness probability_mean': 0.0,
    'Precipitation Total_mean': 0.000694445,
    'Relative Humidity_max': 89.069975,
    'Relative Humidity_min': 49.42117517,
    'Soil Moisture_mean': 0.30982636,
    'Soil Temperature_mean': 22.386598,
    'Sunshine Duration_mean': 21.2153955,
    'Temperature_max': 30.56833333,
    'Temperature_min': 16.38666667,
    'Actual_Disease': 'Pest_E'
}


In [5]:
inputs = [input_1, input_2, input_3]

for idx, input_dict in enumerate(inputs, 1):
    prediction = predict_disease(input_dict, model_name='xgb')
    print(f"\nInput {idx}:")
    print(f"Actual Disease: {input_dict.get('Actual_Disease', 'N/A')}")
    print(f"Predicted Disease: {prediction}")



Input 1:
Actual Disease: Pest_D
Predicted Disease: Pest_D

Input 2:
Actual Disease: Pest_A
Predicted Disease: Pest_A

Input 3:
Actual Disease: Pest_B
Predicted Disease: Pest_B


In [6]:
inputs = [input_1, input_4, input_5]

for idx, input_dict in enumerate(inputs, 1):
    prediction = predict_disease(input_dict, model_name='dl')
    print(f"\nInput {idx}:")
    print(f"Actual Disease: {input_dict.get('Actual_Disease', 'N/A')}")
    print(f"Predicted Disease: {prediction}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step

Input 1:
Actual Disease: Pest_D
Predicted Disease: Pest_D
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step

Input 2:
Actual Disease: Pest_B
Predicted Disease: Pest_B
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step

Input 3:
Actual Disease: Pest_E
Predicted Disease: Pest_E
